In [1]:
#import
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
train_dir = "path_to/train"  # contains 'cats' and 'dogs' folders
test_dir = "path_to/test1"   # test images

# Step 4: Parameters
img_size_hog = (64, 64)      # HOG image resize
img_size_cnn = (224, 224) 

In [ ]:
# Option A: HOG Features
def load_images_hog(folder):
    images, labels = [], []
    for label, cls in enumerate(['cats', 'dogs']):
        cls_folder = os.path.join(folder, cls)
        for file in os.listdir(cls_folder):
            img_path = os.path.join(cls_folder, file)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is not None:
                img = cv2.resize(img, img_size_hog)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                feature = hog(gray, pixels_per_cell=(8,8), cells_per_block=(2,2),
                              orientations=9, block_norm='L2-Hys')
                images.append(feature)
                labels.append(label)
    return np.array(images), np.array(labels)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Load training data with HOG
X_hog, y_hog = load_images_hog(train_dir)
X_train, X_val, y_train, y_val = train_test_split(X_hog, y_hog, test_size=0.2, random_state=42)

# Train SVM
svm_hog = SVC(kernel='linear')
svm_hog.fit(X_train, y_train)

# Evaluate
y_pred = svm_hog.predict(X_val)
print("HOG-SVM Validation Accuracy:", accuracy_score(y_val, y_pred))

In [ ]:
# Option B: CNN Features (VGG16)
cnn_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

def extract_features_cnn(folder):
    features, labels = [], []
    for label, cls in enumerate(['cats', 'dogs']):
        cls_folder = os.path.join(folder, cls)
        for file in os.listdir(cls_folder):
            img_path = os.path.join(cls_folder, file)
            img = image.load_img(img_path, target_size=img_size_cnn)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            feat = cnn_model.predict(x)
            features.append(feat.flatten())
            labels.append(label)
    return np.array(features), np.array(labels)

In [ ]:
# Load CNN features
X_cnn, y_cnn = extract_features_cnn(train_dir)
X_train, X_val, y_train, y_val = train_test_split(X_cnn, y_cnn, test_size=0.2, random_state=42)

# Train SVM
svm_cnn = SVC(kernel='linear')
svm_cnn.fit(X_train, y_train)

# Evaluate
y_pred = svm_cnn.predict(X_val)
print("CNN-SVM Validation Accuracy:", accuracy_score(y_val, y_pred))

In [ ]:
# Predict on Test Set (CNN Example)
def extract_test_features(folder):
    features, filenames = [], []
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = image.load_img(img_path, target_size=img_size_cnn)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feat = cnn_model.predict(x)
        features.append(feat.flatten())
        filenames.append(file)
    return np.array(features), filenames

X_test, test_files = extract_test_features(test_dir)
y_test_pred = svm_cnn.predict(X_test)


In [ ]:
submission = pd.DataFrame({"id": test_files, "label": y_test_pred})
submission.to_csv("submission.csv", index=False)
print("Submission saved as submission.csv")